<a href="https://colab.research.google.com/github/SHEHAN-120/Uni_Handbook_based_RAG/blob/main/Uni_Handbook_based_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install langchain
!pip install langchain_community
!pip install faiss-cpu
!pip install pypdf
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 5.3 MB/s eta 0:00:00


In [ ]:
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
OPEN_API_KEY="sk-prresPUgALlC7BpB0aiNyuZU0-IA"

In [ ]:
from langchain.llms import OpenAI
llm=OpenAI(openai_api_key=OPEN_API_KEY)

/tmp/ipython-input-4-1144433101.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm=OpenAI(openai_api_key=OPEN_API_KEY)


In [ ]:
llm.invoke("Hi Can you translate English to Spanish")

'?\n\nYes, I am an AI language model and I can translate English to Spanish. What would you like me to translate?'

In [ ]:
pdf_reader=PyPDFLoader("/content/Student_Book.pdf")
documents=pdf_reader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=300)
chunks=text_splitter.split_documents(documents)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

embeddings=OpenAIEmbeddings(api_key=OPEN_API_KEY)
db=FAISS.from_documents(documents=chunks,embedding=embeddings)

/tmp/ipython-input-7-1208432781.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings=OpenAIEmbeddings(api_key=OPEN_API_KEY)


In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa=ConversationalRetrievalChain.from_llm(llm=llm,
                                        retriever=db.as_retriever(),
                                        condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                        return_source_documents=True,
                                        verbose=False)


In [ ]:
chat_history=[]
query="""Is that English level 2 passed necessary to got degree"""
result=qa({"question":query,"chat_history":chat_history})
print(result["answer"])

 Yes, according to the context, passing Level I and Level II examinations in English is compulsory in order to obtain a BSc (General) Degree. In the case of a Bachelor of Science (Honours) Degree, a pass in Level III examination is also required in addition to passing Level I and Level II examinations.
